In [ ]:
# import pandas as pd
# import json
# file_path = r'C:\Users\USER\Desktop\DSDE_project\Project\2018\201800002'

In [2]:
import json
import pandas as pd
import os

In [4]:
# for i in range()


# # Load JSON data


def print_nested_json(file_path_x):
    with open(file_path_x, 'r') as file:
        data = json.load(file)

    abstracts_data = data['abstracts-retrieval-response']
    print("Title:",abstracts_data['coredata']['dc:title'])

    print("Publisher:",abstracts_data['coredata']['dc:publisher'])

    print("Publish date:",abstracts_data['coredata']['prism:coverDate'])
  
    indexed_names = [author['preferred-name']['ce:indexed-name'] for author in abstracts_data['authors']['author']]
    print("Authors:", ", ".join(indexed_names))
    key = [key['$'] for key in abstracts_data['authkeywords']['author-keyword']]
    print("keyword:",", ".join(key))
    subjects = [subject['@code'] for subject in abstracts_data['subject-areas']['subject-area']]
    print("Subject:",", ".join(subjects))


In [6]:
def process_json_to_df(file_path):
    try:
        # Read JSON file
        with open(file_path, 'r') as file:
            data = json.load(file)
        
        abstracts_data = data.get('abstracts-retrieval-response', {})
        coredata = abstracts_data.get('coredata', {})
        
        # Helper function to safely get joined values
        def safe_join(data_dict, key_path, subkey=None):
            try:
                if not data_dict.get(key_path):
                    return None
                if subkey:
                    return ", ".join([item.get(subkey, '') for item in data_dict[key_path]]) or None
                return ", ".join(data_dict[key_path]) or None
            except Exception:
                return None

        # Helper function to safely get author names
        def get_author_names():
            try:
                authors = abstracts_data.get('authors', {}).get('author', [])
                if not authors:
                    return None
                names = []
                for author in authors:
                    preferred_name = author.get('preferred-name', {})
                    if preferred_name and 'ce:indexed-name' in preferred_name:
                        names.append(preferred_name['ce:indexed-name'])
                return ", ".join(names) if names else None
            except Exception:
                return None

        # Helper function to safely get keywords
        def get_keywords():
            try:
                keywords = abstracts_data.get('authkeywords', {}).get('author-keyword', [])
                if not keywords:
                    return None
                keyword_list = [key.get('$', '') for key in keywords]
                return ", ".join(keyword_list) if any(keyword_list) else None
            except Exception:
                return None

        # Helper function to safely get subjects
        def get_subjects():
            try:
                subjects = abstracts_data.get('subject-areas', {}).get('subject-area', [])
                if not subjects:
                    return None
                subject_list = [subject.get('@code', '') for subject in subjects]
                return ", ".join(subject_list) if any(subject_list) else None
            except Exception:
                return None
        
        # Create a dictionary with the data
        paper_data = {
            'Title': coredata.get('dc:title') or None,
            'Publisher': coredata.get('dc:publisher') or None,
            'Publish_date': coredata.get('prism:coverDate') or None,
            'Authors': get_author_names(),
            'Keywords': get_keywords(),
            'Subjects': get_subjects()
        }
        
        return paper_data
    
    except Exception as e:
        print(f"Error processing file {file_path}: {str(e)}")
        # Return empty dictionary with None values
        return {
            'Title': None,
            'Publisher': None,
            'Publish_date': None,
            'Authors': None,
            'Keywords': None,
            'Subjects': None
        }

# Rest of your code remains the same
base_path = r'C:\Users\USER\Desktop\DSDE_project\Project\2018\2018'
all_data = []
for i in range(2792):
    file_number = str(i).zfill(5)
    file_path = base_path + file_number
    
    if os.path.exists(file_path):
        paper_data = process_json_to_df(file_path)
        all_data.append(paper_data)
    
    if i % 100 == 0:
        print(f"Processed {i} files...")

df = pd.DataFrame(all_data)

Processed 0 files...
Processed 100 files...
Processed 200 files...
Processed 300 files...
Processed 400 files...
Processed 500 files...
Processed 600 files...
Processed 700 files...
Processed 800 files...
Processed 900 files...
Processed 1000 files...
Processed 1100 files...
Processed 1200 files...
Processed 1300 files...
Processed 1400 files...
Processed 1500 files...
Processed 1600 files...
Processed 1700 files...
Processed 1800 files...
Processed 1900 files...
Processed 2000 files...
Processed 2100 files...
Processed 2200 files...
Processed 2300 files...
Processed 2400 files...
Processed 2500 files...
Processed 2600 files...
Processed 2700 files...


In [7]:
df

""


In [1]:
df.isnull().sum()

NameError: name 'df' is not defined

In [3]:
df.to_csv("2018_data.csv",index=False)

NameError: name 'df' is not defined